In [141]:
import pandas as pd
import numpy as np
import datetime
import os
import time


from dateutil.relativedelta import *
df = pd.read_excel('Laporan Gudang Darurat PMI (s.d. 27 April 2021).xlsx',parse_dates = True).fillna(0)
df = df.iloc[5:,:].reset_index(drop =True)
head = df.iloc[0]
df.columns = head
df = df.iloc[1:,:]
df['Nama Barang'] = df['Nama Barang'].str.capitalize()
df['Berat'] = df['Berat'].replace('0,065', 0.065)
pd.to_numeric(df["Berat"], downcast="float")
df['Total Berat Masuk'] = df['Berat']*df['Masuk'].fillna(0)
df['Total Berat Keluar'] = df['Berat']*df['Keluar'].fillna(0)
df_Berat = pd.pivot_table(df,values = ['Total Berat Masuk','Total Berat Keluar'],
                          index = ['Tanggal','Gudang','Nama Barang'],aggfunc = np.sum).reset_index()
df_Berat['Berat_Balance'] = np.where(df_Berat['Total Berat Masuk'] - df_Berat['Total Berat Keluar']<0,
                                     0,df_Berat['Total Berat Masuk'] - df_Berat['Total Berat Keluar'])
df_Groupbarang = pd.read_excel('Mapping Group Barang.xlsx',parse_dates = True).fillna(0)
df_Groupbarang['Nama Barang'] = df_Groupbarang['Nama Barang'].str.capitalize()
df_Gudangstandard = pd.read_excel('Mapping Gudang Standard.xlsx',parse_dates = True).fillna(0)
df_Berat = df_Berat.merge(df_Groupbarang, how = 'left').fillna("No Group")
df_Berat = df_Berat.merge(df_Gudangstandard, how = 'left').fillna("No Gudang Standard")
df_Berat['Suggestion'] = np.where(df_Berat['Gudang'] == df_Berat['Gudang Standard'],'No relokasi','Relokasi' )
df_Berat = df_Berat.drop(['Total Berat Keluar','Total Berat Masuk'], axis = 1)
df_Berat.columns = ['Tanggal','Gudang Awal','Nama Barang','Berat di Pindah','Group Barang','Gudang Standard','Suggestion']
df_Berat.to_excel('fortableu_wecat.xlsx')

df_Berat

,Tanggal,Gudang Awal,Nama Barang,Berat di Pindah,Group Barang,Gudang Standard,Suggestion
0,2020-02-01,G2,Masker bedah,1980.0,Alkes,G1,Relokasi
1,2020-02-01,G2,Masker n95,0.0,Alkes,G1,Relokasi
2,2020-02-05,G2,Masker bedah,0.0,Alkes,G1,Relokasi
3,2020-02-05,G2,Masker n95,0.0,Alkes,G1,Relokasi
4,2020-02-11,G2,Masker bedah,0.0,Alkes,G1,Relokasi
...,...,...,...,...,...,...,...
3232,2021-04-26,G4,Lay's snack potato 68 g,0.0,Food,G3,Relokasi
3233,2021-04-26,G4,Mie instan,0.0,Food,G3,Relokasi
3234,2021-04-26,G4,Paket karyawan pmi,0.0,Paket,G7,Relokasi
3235,2021-04-27,G4,Sabun marina,0.0,Perlengkapan Hygiene,G2,Relokasi


,Tanggal,Gudang,Nama Barang,Total Berat Keluar,Total Berat Masuk,Berat_Balance,Group Barang,Gudang Standard
0,2020-02-01,G2,Masker Bedah,120.00,2100.0,1980.0,Alkes,G1
1,2020-02-01,G2,Masker N95,32.00,0.0,0.0,Alkes,G1
2,2020-02-05,G2,Masker Bedah,325.20,0.0,0.0,Alkes,G1
3,2020-02-05,G2,Masker N95,40.00,0.0,0.0,Alkes,G1
4,2020-02-11,G2,Masker Bedah,0.60,0.0,0.0,Alkes,G1
...,...,...,...,...,...,...,...,...
3241,2021-04-26,G4,Lay's Snack Potato 68 g,0.00,0.0,0.0,Food,G3
3242,2021-04-26,G4,Mie Instan,0.00,0.0,0.0,Food,G3
3243,2021-04-26,G4,Paket Karyawan PMI,0.00,0.0,0.0,Paket,G7
3244,2021-04-27,G4,Sabun Marina,0.86,0.0,0.0,Perlengkapan Hygiene,G2


In [77]:
df

array(['Total Berat Masuk', 'Total Berat Keluar'], dtype=object)